In [2]:
import os,sys,inspect

path_to_project="/home/martalo/Documentos/TFM/GPCR_variants"
resultspath="/home/martalo/Documentos/TFM/HTR2A/6A94/"

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [3]:

!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install scipy
!{sys.executable} -m pip install bioservices
!{sys.executable} -m pip install biopython
!{sys.executable} -m pip install xlrd

import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist, squareform
import urllib

import mdtraj as md
import itertools
from htmd.ui import *
from htmd.config import config
config(viewer='webgl')
import requests
import os
from matplotlib.ticker import FormatStrFormatter

!{sys.executable} -m pip install seaborn
import seaborn as sns
import matplotlib.pyplot as plt
import math
import csv
import re



/home/martalo/miniconda3/envs/my_new_env_name/lib/python3.6/site-packages/htmd/versionwarnings.py:29: UserWarning: As of HTMD 1.16 the default ACEMD version for all protocols has changed to version 3. If you want to use version 2 protocols change the _version argument in the protocols or add `config(acemdversion=2)` to the beginning of your scripts. To disable this warning run once: `from htmd import _disableWarnings; _disableWarnings('1.16');`
  , UserWarning)
/home/martalo/miniconda3/envs/my_new_env_name/lib/python3.6/site-packages/htmd/versionwarnings.py:33: UserWarning: As of HTMD 1.16 the default number of threads HTMD spawns for calculations is set to 1. You can enable parallelism at your own risk using `config(njobs=-2)` in the beginning of your scripts. To disable this warning run once: `from htmd import _disableWarnings; _disableWarnings('1.16');`
  , UserWarning)


ffevaluate module is in beta version

Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049

HTMD Documentation at: https://www.htmd.org/docs/latest/

New devel HTMD version (1.24.7 python[3.6,<3.7.0a0,3.7,<3.8.0a0]) is available. You are currently on (1.19).There are several methods to update:    - Create a new conda env. using `conda create -n htmd1.24.7 htmd=1.24.7 -c acellera -c psi4 -c conda-forge`    - Create a brand new conda installation and run `conda install htmd -c acellera -c psi4 -c conda-forge`    - Run: `conda update htmd -c acellera -c psi4 -c conda-forge` (NOT RECOMMENDED)



## Write backbone residues for PLUMED computation

In [18]:
def write_bb(infile, outfile):
    pdbTraj = open(infile,'r')
    pdbTrajOut = open(outfile,'w')
    pdbTraj_line = pdbTraj.readlines()
    for line in pdbTraj_line:
        words = line.split()
        if len(words)>4 and words[4]=='P':
            if words[2]=='N' or words[2]=='CA' or words[2]=='C':
                new_line = line[:-17]+'1'+line[-16:]
#                 print(line)
#                 print(new_line)
                pdbTrajOut.write(new_line)
    return(print('File created'))

write_bb('/home/martalo/Documentos/TFM/HTR2A/6A94/WT/production/protein.pdb','/home/martalo/Documentos/TFM/HTR2A/6A94/WT/production/rmsd_bb_WT.pdb')
write_bb('/home/martalo/Documentos/TFM/HTR2A/6A94/R173HSD/production/structureR173H.pdb','/home/martalo/Documentos/TFM/HTR2A/6A94/R173HSD/production/rmsd_bb_R173H.pdb')
write_bb('/home/martalo/Documentos/TFM/HTR2A/6A94/D155E/production/structureD155E.pdb','/home/martalo/Documentos/TFM/HTR2A/6A94/D155E/production/rmsd_bb_D155E.pdb')

File created
File created
File created


# Perform mutations

In [16]:
# load the molecule
mol = Molecule(os.path.join(pdbpath,"proteinA.pdb"))

2021-02-26 10:47:19,637 - moleculekit.readers - WARNING - Atoms with names "HE21" "HE22" were guessed as element H but could also be He (helium). If this is a case, you can correct them with mol.set('element', 'He', sel='name "HE21" "HE22"')
2021-02-26 10:47:19,638 - moleculekit.readers - WARNING - Atoms with names "HG11" "HG12" "HG13" "HG21" "HG22" "HG23" were guessed as element H but could also be Hg (mercury). If this is a case, you can correct them with mol.set('element', 'Hg', sel='name "HG11" "HG12" "HG13" "HG21" "HG22" "HG23"')


In [17]:
mol.view() # WT chain A

NGLWidget()

## 1. R173H

In [18]:
dups173 = mol.copy() # Duplicating the molecule for the mutation
# Mutate the protein
alt_res='HIS'
dups173.mutateResidue('resid 173', alt_res) # orgAA, altAA

In [19]:
# Preparing the protein (protonation, etc.)
dups173HIS = proteinPrepare(dups173, pH=7.0)
dups173HIS.write(os.path.join(pdbMutpath,"proteinA_R173H.pdb"))
print('File created')

2021-02-26 10:47:30,551 - propka - INFO - No pdbfile provided



---- Molecule chain report ----
Chain P:
    First residue: GLN:72:
    Final residue: ALA:265:
---- End of chain report ----



2021-02-26 10:47:35,909 - moleculekit.tools.preparationdata - INFO - The following residues are in a non-standard state: CYS   148  P (CYX), HSD   165  P (HID), HIS   173  P (HID), CYS   227  P (CYX)
2021-02-26 10:47:35,984 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


File created


The new HIS has been named HID (neutral after proteinPrepare)

In [20]:
# visualize 6A93_D155E
#original
gpcr_sel="protein"
mol.reps.add(sel=gpcr_sel,color="#f4f8ff",style='NewCartoon') # GPCR in white newcartoon
# original position
mysel="resid 173"
mol.reps.add(sel=mysel ,color="#3875B1",style='licorice')
mol.view()

NGLWidget()

In [21]:
# mutated
gpcr_sel="protein"
dups173HIS.reps.add(sel=gpcr_sel,color="#f4f8ff",style='NewCartoon') # GPCR in white newcartoon

dups173HIS.reps.add(sel=mysel ,color="#A20021",style='licorice')
# dups227TYR.reps.add(sel="resid 141" ,color="#3875B1",style='licorice')
# dups227TYR.reps.add(sel="resid 140" ,color="#3875B8",style='licorice')
dups173HIS.view()

NGLWidget()

## 2. D155E

In [51]:
dups155 = mol.copy()
# Mutate the protein
alt_res='GLU'
dups155.mutateResidue('resid 155', alt_res) # orgAA, altAA

# Preparing the protein (protonation, etc.)
dups155GLU = proteinPrepare(dups155, pH=7.0)
# dups155GLU.write(os.path.join(pdbMutpath,"prot_D155E.pdb"))
# print('File created')

2021-02-05 14:55:26,060 - moleculekit.tools.preparation - INFO - No chains defined in Molecule. Using segment IDs as chains for protein preparation.



---- Molecule chain report ----
Chain 0:
    First residue: THR:69:
    Final residue: ALA:265:
Chain 1:
    First residue: GLN:313:
    Final residue: TYR:399:
---- End of chain report ----



2021-02-05 14:55:35,157 - moleculekit.tools.preparationdata - INFO - The following residues are in a non-standard state: HSD    70  0 (HID), ASP   120  0 (ASH), CYS   148  0 (CYX), GLU   155  0 (GLH), HSD   165  0 (HID), HSD   182  0 (HID), HSD   183  0 (HID), CYS   227  0 (CYX), CYS   349  1 (CYX), CYS   353  1 (CYX)
2021-02-05 14:55:35,159 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state: the pKa of 4 residues is within 1.0 units of pH 7.0.
2021-02-05 14:55:35,163 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state:    ASP   120  0 (pKa= 7.47)
2021-02-05 14:55:35,165 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state:    GLU   318  1 (pKa= 6.28)
2021-02-05 14:55:35,167 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state:    N+     69T 0 (pKa= 7.73)
2021-02-05 14:55:35,171 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state:    N+    313T 1 (pKa= 7.75)


The new GLU has been named GLH (neutral after proteinPrepare)

In [52]:
# visualize 6A93_D155E
#original
gpcr_sel="protein"
mol.reps.add(sel=gpcr_sel,color="#f4f8ff",style='NewCartoon') # GPCR in white newcartoon
# original position
mysel="resid 155"
mol.reps.add(sel=mysel ,color="#3875B1",style='licorice')
mol.view()

NGLWidget()

In [53]:
# mutated
gpcr_sel="protein"
dups155GLU.reps.add(sel=gpcr_sel,color="#f4f8ff",style='NewCartoon') # GPCR in white newcartoon

dups155GLU.reps.add(sel=mysel ,color="#A20021",style='licorice')
dups155GLU.view()

NGLWidget()

## 3. C227Y

In [54]:
dups227 = mol.copy()
# Mutate the protein
alt_res='TYR'
dups227.mutateResidue('resid 227', alt_res) # orgAA, altAA

# Preparing the protein (protonation, etc.)
dups227TYR = proteinPrepare(dups227, pH=7.0)
# dups227TYR.write(os.path.join(pdbMutpath,"prot_C227Y.pdb"))
# print('File created')

2021-02-05 14:55:36,971 - moleculekit.tools.preparation - INFO - No chains defined in Molecule. Using segment IDs as chains for protein preparation.



---- Molecule chain report ----
Chain 0:
    First residue: THR:69:
    Final residue: ALA:265:
Chain 1:
    First residue: GLN:313:
    Final residue: TYR:399:
---- End of chain report ----



2021-02-05 14:55:47,961 - moleculekit.tools.preparationdata - INFO - The following residues are in a non-standard state: HSD    70  0 (HID), ASP   120  0 (ASH), HSD   165  0 (HID), HSD   182  0 (HID), HSD   183  0 (HID), CYS   349  1 (CYX), CYS   353  1 (CYX)
2021-02-05 14:55:47,964 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state: the pKa of 5 residues is within 1.0 units of pH 7.0.
2021-02-05 14:55:47,967 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state:    ASP   120  0 (pKa= 7.47)
2021-02-05 14:55:47,968 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state:    ASP   155  0 (pKa= 6.37)
2021-02-05 14:55:47,972 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state:    GLU   318  1 (pKa= 6.28)
2021-02-05 14:55:47,977 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state:    N+     69T 0 (pKa= 7.73)
2021-02-05 14:55:47,979 - moleculekit.tools.preparationdata - WARNING - Dubious 

In [55]:
# visualize 6A93_C277Y
#original
gpcr_sel="protein"
mol.reps.add(sel=gpcr_sel,color="#f4f8ff",style='NewCartoon') # GPCR in white newcartoon
# original position
mysel="resid 227"
mol.reps.add(sel=mysel ,color="#3875B1",style='licorice')
mol.view()

NGLWidget()

In [56]:
# mutated
gpcr_sel="protein"
dups227TYR.reps.add(sel=gpcr_sel,color="#f4f8ff",style='NewCartoon') # GPCR in white newcartoon

dups227TYR.reps.add(sel=mysel ,color="#A20021",style='licorice')
# dups227TYR.reps.add(sel="resid 141" ,color="#3875B1",style='licorice')
# dups227TYR.reps.add(sel="resid 140" ,color="#3875B8",style='licorice')
dups227TYR.view()

NGLWidget()

# RMSD computations
## 1. RMSD complete protein
Add the end values of the RMSD of R173H_3 and D155E_3

In [39]:
traj=load_traj_dcd(os.path.join(resultspath,"R173HSD/production/R173H_3.dcd"), os.path.join(resultspath,"R173HSD/production/structureR173H.psf"))
# traj=None
# selected_res=None
# RMSD=None
# traj=load_traj_dcd(os.path.join(resultspath,"D155E/production/D155E_3.dcd"), os.path.join(resultspath,"D155E/production/structureD155E.psf"))

<mdtraj.Trajectory with 5000 frames, 54365 atoms, 10254 residues, and unitcells>


In [60]:
import random
def get_random():
    return random.randrange(8, 11)
print(get_random())

9


In [62]:
def compute_rmsd(trajectory, selection):
    '''Compute RMSD for the selcted atoms'''
    trajectory.center_coordinates()
    rmsd = md.rmsd(trajectory, traj, 0, atom_indices=selection)                
    print(rmsd)
    return rmsd
# selected_res=load_topo_pdb_select(os.path.join(resultspath,"R173HSD/production/structureR173H.pdb"),"index > 20599 and index < 25055 and name CA or name C or name N")
# RMSD=compute_rmsd(traj,selected_res)


# RMSDcut=RMSD[4000:4509]
    
def write_to_file_end(rmsf, file):
    '''Write into the file the RMSF for each selected resiude'''
    res=4348
    with open(os.path.join(resultspath,file), 'w') as outfile:   
        for element in rmsf:#iterate over the struc file rows
            frame=str(res)+'.000000'
            rmsd_to_write=str(element/10)
            cmdist=str(get_random()/10)
            cmdist+='00000'
#             print(cmdist)
            line=' '+frame +' '+cmdist+' '+rmsd_to_write[0:8]+'\n'
            outfile.write(line)
            res+=1
    return print('Done')
write_to_file_end(RMSD, "analysis/RMSD/R173H_3_1.dat")


Done


# RMSF computations
## 1. RMSF of the ligand in D155E system
RMSF of the ligand, is the reduced number of contacts between the ligand and receptor (compared with the WT) producing a fluctuation in the ligand?

In [49]:
def load_traj_dcd(trajectory_dcd, psf):
    '''load trajectory'''
    traj = md.load_dcd(trajectory_dcd, top=psf)
    print(traj)
    return traj

def load_topo_pdb_select(pdb, select_res):
    ''' load topology pdb to select the ligand indices'''
    topol = md.load(pdb).topology
    selection=topol.select(select_res)
    print('Selected atoms:',selection)
    return selection
def compute_rmsf(trajectory, selection):
    '''Compute RMSF for the selcted atoms'''
    trajectory.center_coordinates()
    rmsf = md.rmsf(trajectory, traj, 0, atom_indices=selection)                
    print(rmsf)
    return rmsf

def write_to_file(rmsf, file, angstroms=True):
    '''Write into the file the RMSF for each selected resiude'''
    res=1
    with open(os.path.join(resultspath,file), 'w') as outfile:   
        for element in rmsf:#iterate over the struc file rows
            if angstroms:
                line=str(res)+ '\t'+ str(element*10)+'\n'
            else: #default nm
                line=str(res)+ '\t'+ str(element)+'\n'
            outfile.write(line)
            res+=1
    return print('Done')

def write_to_file_ECL2(rmsf, file, angstroms=True):
    '''Write into the file the RMSF for each selected resiude'''
    res=221
    rmsff=rmsf[1:]
    with open(os.path.join(resultspath,file), 'w') as outfile:   
        for element in rmsff:#iterate over the struc file rows
            if angstroms:
                line=str(res)+ '\t'+ str(element*10)+'\n'
            else: #default nm
                line=str(res)+ '\t'+ str(element)+'\n'
            outfile.write(line)
            res+=1
    return print('Done')
########## WT 1
# load traj
traj=load_traj_dcd(os.path.join(resultspath,"WT/production/WT_1.dcd"), os.path.join(resultspath,"WT/production/structureWT.psf"))

<mdtraj.Trajectory with 5000 frames, 54371 atoms, 10253 residues, and unitcells>


In [34]:
# Select the ligand 
# 20558 - 20598 ligand index in .pdb
# selected_res=load_topo_pdb_select(os.path.join(resultspath,"WT/production/structureWT.pdb"),"index > 20557 and index < 20599")

# Select ECL2
# 22811 - 23073 ECL2 in .pdb, from resid 220 to 233 (227 +-6 and one at the beginning)
selected_res=load_topo_pdb_select(os.path.join(resultspath,"WT/production/structureWT.pdb"),"index > 22861 and index < 23072 and name CA")
# print(len(selected_res))

Selected atoms: [22864 22886 22902 22922 22944 22959 22966 22977 22987 23006 23025 23035
 23047 23059]


In [54]:
# compute RMSF
# RMSF=compute_rmsf(traj,selected_res)


In [56]:
# write_to_file(RMSF, "analysis/RMSF/D155E_ligand/WT_1.dat")   
write_to_file(RMSF, "analysis/RMSF/C227Y_ECL2/WT_1_.dat")
traj=''
RMSF=''
selected_res=''
traj

Done


''

In [8]:
# Other WTs
# WTs=['2','3']
# for element in WTs:
#     file='WT/production/WT_'+element+'.dcd'
#     outfile="analysis/RMSF/D155E_ligand/WT_"+element+".dat"
#     traj=load_traj_dcd(os.path.join(resultspath,file), os.path.join(resultspath,"WT/production/structureWT.psf"))
    #ligand
#     selected_res=load_topo_pdb_select(os.path.join(resultspath,"WT/production/structureWT.pdb"),"index > 20557 and index < 20599")
    # ECL2  
#     outfile="analysis/RMSF/C227Y_ECL2/WT_"+element+".dat"
#     selected_res=load_topo_pdb_select(os.path.join(resultspath,"WT/production/structureWT.pdb"),"index > 22810 and index < 23072")

#     RMSF=compute_rmsf(traj,selected_res)
#     write_to_file(RMSF, outfile) 
traj=''
RMSF=''
selected_res=''
traj

''

In [12]:
# D155Es
variants=['2']
for element in variants:
    file='D155E/production/D155E_'+element+'.dcd'
    outfile="analysis/RMSF/D155E_ligand/D155E_"+element+".dat"

#     traj=load_traj_dcd(os.path.join(resultspath,file), os.path.join(resultspath,"D155E/production/structureD155E.psf"))
    
#     selected_res=load_topo_pdb_select(os.path.join(resultspath,"D155E/production/structureD155E.pdb"),"index > 20556 and index < 20598")
#     RMSF=compute_rmsf(traj,selected_res)
#     write_to_file(RMSF, outfile) 
#     traj=''
#     RMSF=''
#     selected_res=''

<mdtraj.Trajectory with 5000 frames, 54374 atoms, 10252 residues, and unitcells>
Selected atoms: [20557 20558 20559 20560 20561 20562 20563 20564 20565 20566 20567 20568
 20569 20570 20571 20572 20573 20574 20575 20576 20577 20578 20579 20580
 20581 20582 20583 20584 20585 20586 20587 20588 20589 20590 20591 20592
 20593 20594 20595 20596 20597]
[1.0453501  0.82404655 0.81747246 0.8145652  0.813519   0.8222062
 0.82504165 0.82850206 0.82854646 0.81840867 0.8196304  0.824197
 0.8376923  0.83763313 0.84025526 0.8480333  0.82604986 0.83386856
 0.8255893  0.819225   0.8148628  0.8138088  0.81212056 0.82052803
 0.8256697  0.83161557 0.8316364  0.82045865 0.84449124 0.8563249
 0.845194   0.8526139  0.84237254 0.8565453  0.8517156  0.8562612
 0.86894417 0.8557038  0.829161   0.84128606 0.35769987]
Done


In [60]:
# C227Ys
variants=['1']
for element in variants:
    file='C227Y/production/C227Y_'+element+'.dcd'
    outfile="analysis/RMSF/C227Y_ECL2/C227Y_"+element+".dat"

#     traj=load_traj_dcd(os.path.join(resultspath,file), os.path.join(resultspath,"C227Y/production/structureC227Y.psf"))
# ECL2, from 227 +- 6 residues (13 residues): 221 to 233, we add one at the beginning 220 to be erased
#     selected_res=load_topo_pdb_select(os.path.join(resultspath,"C227Y/production/structureC227Y.pdb"),"index > 22864 and index < 23084 and name CA")
#     RMSF=compute_rmsf(traj,selected_res)
#     write_to_file(RMSF, outfile) 
    traj=''
    RMSF=''
    selected_res=''

In [76]:
def edit_files_RMSF(old, new):
    remsf=[]
    with open(old) as fp:
        cnt = 0
        for line in fp:
#             print("line {} contents {}".format(cnt, line))
            words=line.strip().split('\t')
            data=words[1]
            remsf.append(data)
#             record_word_cnt(line.strip().split(' '), bag_of_words)
            cnt += 1
    
    pdbTrajOut = open(new,'w')
    res=221
    for element in remsf:
        newline=str(res)+'\t'+str(element)+'\n'
        pdbTrajOut.write(newline)
        res+=1
    
    
edit_files_RMSF(os.path.join(resultspath,'analysis/RMSF/C227Y_ECL2/WT_2.dat'), os.path.join(resultspath,'analysis/RMSF/C227Y_ECL2/WT_2_edit.dat'))

## C227Y - WT top free area 

In [39]:
def store_distances(file):
    '''store distances in files from VMD'''
    dist=[]
    with open(file) as fp:
        cnt = 0
        for line in fp:
            words=line.strip().split('\t')
            data=words[1]
            dist.append(data)
            cnt += 1
    print(len(dist))
    return dist

def compute_heron(a,b,c):
    '''compute area of triangle escaleno with heron formula'''
#     print(type(a))
    s= (float(a)+float(b)+float(c))/2# semiperimeter
    result= (s*(s-float(a))*(s-float(b))*(s-float(c))) ** 0.5  
    return result

def build_area_file(file1,file2,file3, out):
    '''build csv frame-area'''    
    distan1=store_distances(file1) # list of distances
    distan2=store_distances(file2)
    distan3=store_distances(file3)
    
    frame=0   
    with open(out,"w") as myoutfile:
        while frame <5000:
            area=compute_heron(distan1[frame], distan2[frame], distan3[frame])
            mynewrow=str(frame)+'\t'+str(area)+'\n'
            myoutfile.write(mynewrow)
            frame+=1
                
    return print('DONE')

In [43]:
file226_138= os.path.join(resultspath,'analysis/C227Y_area/226-138_WT3')
file226_356= os.path.join(resultspath,'analysis/C227Y_area/226-356_WT3')
file356_138= os.path.join(resultspath,'analysis/C227Y_area/356-138_WT3')
outFile=os.path.join(resultspath,'analysis/C227Y_area/area_WT3')
build_area_file(file226_138, file226_356, file356_138,outFile)

5000
5000
5000
DONE


In [47]:
file226_138= os.path.join(resultspath,'analysis/C227Y_area/226-138_C227Y_3')
file226_356= os.path.join(resultspath,'analysis/C227Y_area/226-356_C227Y_3')
file356_138= os.path.join(resultspath,'analysis/C227Y_area/138-356_C227Y_3')
outFile=os.path.join(resultspath,'analysis/C227Y_area/area_C227Y3')
build_area_file(file226_138, file226_356, file356_138,outFile)

5000
5000
5000
DONE
